In [4]:
import requests
from bs4 import BeautifulSoup

In [400]:
class OsrsPlayer:
    def __init__(self, username):
        # Gather parameters using class functions
        self.username = self.__fix_username(username) # Prepare username for request
        self.lowest_skill = ["",99]
        self.lowest_xp = ["",200000000]
        self.highest_xp = ["",0]
        self.lvl99 = [] # List of 99's

        # Html Get request
        self.__html = BeautifulSoup( requests.get(f"https://secure.runescape.com/m=hiscore_oldschool/hiscorepersonal?user1={self.username}").text, 'html.parser')

        # Check if player exists.
        try:
            player_exist = (False if "No player" in self.__html.find("div", {"align":"center"}).find("div", {"align":"center"}).text else False)
            if player_exist == False:
                print("Player Not Found!")
        except:
            # Run functions to overwrite default values
            self.__skill_stats = self.__collect_skills()
            self.__clue_stats = self.__collect_clues()
            self.__other_stats = self.__collect_other()
            print("Player Stats Loaded!")
            
    # Used in __init__ to replace spaces with % to work in link request
    def __fix_username(self, username):
        try:
            username.replace(" ","%20")
        except:
            pass
        return username
    
    def __collect_skills(self):
        self.__skill_stats = {}

        # Get overall stat if exists
        try:
            overall_slice = self.__html.find_all("td", {"align" : "right"})[3:6]
            overall_rank = int(overall_slice[0].text.replace(",",""))
            overall_level = int(overall_slice[1].text.replace(",",""))
            overall_xp = int(overall_slice[2].text.replace(",",""))
            self.__skill_stats["Overall"] = [overall_rank, overall_level, overall_xp]
            skills = self.__html.find_all("td", {"align" : "right"})[6:] # Define slice containing Skills
        except:
            self.__skill_stats["Overall"] = ["Unknown", "Unknown", "Unknown"] # Unknown if user has no overall stat
            skills = self.__html.find_all("td", {"align" : "right"})[3:] # Define slice containing Skills if user does not have overall stat

        # Loop over each skill and get rank, level and xp, appending this to self.skill_stats
        for skill_nr in range(23):
            current_skill_slice = skills[skill_nr * 4 : skill_nr * 4 + 4]

            # Check if current image urr has 1.gif in it (which means it is a skill)
            if "1.gif" in current_skill_slice[0].find("img")["src"]:

                # Gather information from slice
                current_skill_icon = current_skill_slice[0].find("img")["src"]
                current_skill = current_skill_slice[0].find("img")["src"].split("_")[-1].replace("1.gif","").capitalize()
                current_skill_rank = int(current_skill_slice[1].text.replace(",","")) # Rank
                current_skill_level = int(current_skill_slice[2].text.replace(",","")) # Level
                current_skill_xp = int(current_skill_slice[3].text.replace(",","")) # XP
                
                # Find Lowest Skill
                if current_skill_level < self.lowest_skill[1]:
                    self.lowest_skill[0] = current_skill
                    self.lowest_skill[1] = current_skill_level
                    
                # Find Lowest XP
                if current_skill_xp < self.lowest_xp[1]:
                    self.lowest_xp[0] = current_skill
                    self.lowest_xp[1] = current_skill_xp
                    
                # Count 99's
                if current_skill_level == 99:
                    self.lvl99.append(current_skill)
                if len(self.lvl99) == 23:
                    self.lvl99 = "Max"
                    
                # Get Highest XP
                if current_skill_xp > self.highest_xp[1]:
                    self.highest_xp[0] = current_skill
                    self.highest_xp[1] = current_skill_xp
    
                # Append skill, rank, level and xp to self.skill_stats
                self.__skill_stats[current_skill] = [current_skill_rank,current_skill_level,current_skill_xp, current_skill_icon]
            else:
                break
        return self.__skill_stats

    def __collect_clues(self):
        self.__clue_stats = {}

        # Loop over all td elements in the html document
        for index, element in enumerate(self.__html.find_all("td", {"align" : "right"})):
            try:
                # If the element has a link which contains the word "clue"
                # We know what index to start gathering clue information
                if "clue" in element.find("img")["src"]:
                    clue_start_index = index # Break when finding first element with clue in url
                    break
            except:
                continue 

        # Create html slice of elements from where we found the clue start index
        clues = self.__html.find_all("td", {"align" : "right"})[clue_start_index:]
        
        for clue_nr in range(7):
            current_clue_slice = clues[clue_nr * 3 : clue_nr * 4 + 3]
            try:
                if "clue" in current_clue_slice[0].find("img")["src"]:
                    current_clue_icon = current_clue_slice[0].find("img")["src"] # Clue icon url
                    current_clue = current_clue_slice[0].find("img")["src"].split("_")[-1].replace(".png","")[11:].capitalize() # Clue Name
                    
                    current_clue_rank = int(current_clue_slice[1].text.replace(",","")) # Rank
                    current_clue_score = int(current_clue_slice[2].text.replace(",","")) # Score
                    
                    self.__clue_stats[current_clue] = [current_clue_rank, current_clue_score, current_clue_icon]
            
                else:
                    continue
            except:
                break
        return self.__clue_stats


    def __collect_other(self):
        self.__other_stats = {}

        # Loop over all td elements in the html document
        for index, element in enumerate(self.__html.find_all("td", {"align" : "right"})):
            try:
                # If the element has a link which contains the word "clue"
                # We get the last element in the clue list, where we start gathering other stats
                if "clue" in element.find("img")["src"]:
                    other_start_index = index + 3
            except:
                continue 
    
        # Create html slice of elements from where we found the clue start index
        other = self.__html.find_all("td", {"align" : "right"})[other_start_index:]
        try:
            for other_nr in range(100):
                current_other_slice = other[other_nr * 3 : other_nr * 4 + 3]
    
                current_other_icon = current_other_slice[0].find("img")["src"] # Clue icon url
                current_other = current_other_slice[0].find("img")["src"].split("_")[-1].replace(".png","")[:].capitalize() # Clue Name
                
                current_other_rank = int(current_other_slice[1].text.replace(",","")) # Rank
                current_other_score = int(current_other_slice[2].text.replace(",","")) # Score
     
                self.__other_stats[current_other] = [current_other_rank, current_other_score, current_other_icon]
        except:
            pass
        return self.__other_stats

    # Return Rank of spesific Skill
    def rank(self, skill):
        try:
            return self.__skill_stats[skill][0]
        except:
            return f"Unable to return {skill} rank."
     
    # Return Level of spesific Skill
    def level(self, skill):
        try:
            return self.__skill_stats[skill][1]
        except:
            return f"Unable to return {skill} level."
    
    # Return XP in spesific skill 
    def xp(self, skill):
        try:
            return self.__skill_stats[skill][2]
        except:
            return f"Unable to return {skill} xp."

    # Display All Stats
    def skills(self, raw=False):
        if raw == True:
            return self.__skill_stats
        else:
            print("                 |     Rank     |     Level    |      XP")
            for skill in self.__skill_stats:
                if "Unknown" in self.__skill_stats[skill]:
                    continue
                skill_rank = f"{self.__skill_stats[skill][0]:,d}" # Add thousand delimeters
                skill_level = f"{self.__skill_stats[skill][1]:,d}" # Add thousand delimeters
                skill_xp = f"{self.__skill_stats[skill][2]:,d}" # Add thousand delimeters
                print(f"{skill:^15}  |  {skill_rank:^10}  |  {skill_level:^10}  |  {skill_xp:^10}")

    # Display all clue stats
    def clues(self, raw=False):
        if raw == True:
            return self.__clue_stats
        else:
            print("                 |     Rank     |    Score")
            for clue in self.__clue_stats:
                clue_rank = f"{self.__clue_stats[clue][0]:,d}"
                clue_score = f"{self.__clue_stats[clue][1]:,d}"
                print(f"{clue:^15}  |  {clue_rank:^10}  |  {clue_score:^10}")

    def other(self, raw=False):
        if raw == True:
            return self.__other_stats
        else:
            print("                      |     Rank     |    Score")
            for other in self.__other_stats:
                other_rank = f"{self.__other_stats[other][0]:,d}"
                other_score = f"{self.__other_stats[other][1]:,d}"
                print(f"{other:^20}  |  {other_rank:^10}  |  {other_score:^10}")
                
    

In [415]:
kvarme = OsrsPlayer("Solo Kvarme")

Player Stats Loaded!


In [423]:
kvarme.skills(raw=False)

                 |     Rank     |     Level    |      XP
    Overall      |   278,521    |    1,996     |  146,017,744
    Attack       |   445,679    |      93      |  7,502,794 
    Defence      |   514,927    |      90      |  5,359,831 
   Strength      |   398,224    |      99      |  13,211,687
   Hitpoints     |   303,768    |      99      |  18,884,104
    Ranged       |   339,520    |      99      |  14,606,142
    Prayer       |   475,635    |      78      |  1,710,733 
     Magic       |   232,804    |      99      |  13,987,891
    Cooking      |   558,777    |      86      |  3,864,476 
  Woodcutting    |   934,364    |      72      |   987,443  
   Fletching     |   943,589    |      71      |   850,408  
    Fishing      |   297,294    |      88      |  4,736,036 
  Firemaking     |    67,646    |      99      |  13,835,985
   Crafting      |   271,385    |      88      |  4,517,419 
   Smithing      |   316,570    |      83      |  2,898,528 
    Mining       |   473,49

In [419]:
kvarme.clues(raw=False)

                 |     Rank     |    Score
      All        |   162,599    |     302    
     Easy        |    60,965    |      87    
    Medium       |   110,611    |     124    
     Hard        |   234,764    |      86    
     Elite       |   352,537    |      5     


In [420]:
kvarme.other(raw=False)

                      |     Rank     |    Score
    Riftsclosed       |   177,575    |     115    
    Abyssalsire       |   212,947    |      10    
   Barrowschests      |    75,488    |     539    
  Chambersofxeric     |   170,288    |      40    
   Chaoselemental     |   177,018    |      10    
  Commanderzilyana    |   242,356    |      7     
 Crazyarchaeologist   |   118,417    |      50    
    Dagannothrex      |   206,195    |     168    
      Hespori         |   167,884    |      51    
       Kraken         |   204,291    |    1,100   
     Sarachnis        |   137,730    |      50    
      Skotizo         |   299,938    |      14    
     Tempoross        |   395,318    |      28    
    Thegauntlet       |    91,818    |      29    
Thecorruptedgauntlet  |   115,588    |      67    
    Thewhisperer      |    34,769    |      5     
   Tombsofamascut     |   101,865    |      35    
      Tztokjad        |    92,746    |      10    
      Vorkath         |   412,303 